(basics-model)=
# Basic oemof-solph energy system model

This tutorial shows you how to use the basic components of oemof-solph to build your own simple linear energy system model. A similiar introduction to the basics concept can also be found in the [documentation](https://oemof-solph.readthedocs.io/en/stable/basic_concepts/energy_system.html#) of oemof-solph. In the following we will draw heavily from this explanation. 

We will use the example of the energy system of a house, with electricity and heating demand, a roof pv-system, and a connection to the electrical and gas grid.

![energy system](../figures/energy_system.png)

## Build the energy system

The first step to build your energy system is using the class `solph.EnergySystem()`. The `solph.EnergySystem()` instance will contain all of the model’s components.
`solph.EnergySystem()` requires you to give it the time resolution over which your model is defined by providing a time index, usually in form of a `pandas.DatetimeIndex`. (But a simple list of floats would also works as long as it is increasing in value.)

In [32]:
import oemof.solph as solph
import pandas as pd

# Define the time index of the model
timesteps = pd.date_range(start='2025-01-01 00:00:00', 
                          end='2025-12-31 23:00:00', 
                          freq='h')

print(f"Total timesteps: {len(timesteps):,} (Hourly from {timesteps[0]} to {timesteps[-1]})\n")

Total timesteps: 8,760 (Hourly from 2025-01-01 00:00:00 to 2025-12-31 23:00:00)



To create your *EnergySystem* you have to pass the time index at initialisation. `solph.EnergySystem()` has some more key word arguments, but for now, only one is important: `infer_last_intervall=False`. This key word argument sepcifies if you time horizon ends at the last time point you specified in your time index or if this time point is the beginning of the last time step. Setting it to `True` only works if your time index is equidistant, i.e. all time steps have the same duration.

In [33]:
# Build an energy system with infer_last_interval = False
energy_system_ = solph.EnergySystem(
    timeindex=timesteps, infer_last_interval=False
)


# Build an energy system with infer_last_interval = True
energy_system = solph.EnergySystem(
    timeindex=timesteps, infer_last_interval=True
)

# Show the difference in the time index
for i in [energy_system_, energy_system]:
    print(
        f"Time index when infer_last_intervall=",
        f"{True if i==energy_system else False}:\n {len(i.timeindex):,}", 
        f"(Hourly from {timesteps[0]} to {i.timeindex[-1]})\n"
    )

Time index when infer_last_intervall= False:
 8,760 (Hourly from 2025-01-01 00:00:00 to 2025-12-31 23:00:00)

Time index when infer_last_intervall= True:
 8,761 (Hourly from 2025-01-01 00:00:00 to 2026-01-01 00:00:00)



For an in depth discussion of time index in oemof-solph refer to ...

## Energy System Elements

Next we add elements to this *EnergySystem*. 

In general, the *EnergySystem* is a mathematical graph of edges and nodes, represented by three main elements in oemof.solph:
- Nodes: Buses and Components
- Edges: Flows 

### Buses

You usually start with the *Buses*, since they will be referenced by the other compontens. The main purpose of a *Bus* is the balancing of its inflows and its outflows at any given point in time, meaning, that the sum of all incoming flows and all outgoing flows must be equal to zero.

To add a bus to the *EnergySystem*, all you have to do is create an instance of the `solph.Bus()` class, give it a name (aka label) and add it to the energy system. The label has to be unique. If you do not set a label, a random label is assigned, but this makes it difficult to track the results later on. The label and the python variable name do not have to match. You use the variable name in the code to refer to the object and add it to the energy system, and you can use the label when processing the results.

Then all you need to do is add the bus to you `solph.EnergySystem()` instance.

```{tip}
A typical source of error is forgetting to add your components to the energy system. The error message reads quite cryptic such as ... . 
```

In [34]:
# Define three buses for the commodities electricity, heat and natural gas
el_bus = solph.Bus(label = "electric bus")
heat_bus = solph.Bus(label = 'heat bus')
gas_bus = solph.Bus(label='gas bus')

# you can add Buses individuall or all together
energy_system.add(el_bus)
energy_system.add(heat_bus, gas_bus)

### Components

The components are the core of the underlying network of the energy system. All component can be assigned a user-label when created. The user has to provide information regarding connections between busses and in/out of the component. A component which has only inputs defined is referred to as a sink. A component with only outputs is referred to as a source. A component with both input and outputs is a converter. These three very basic type of components are provided by the classes `Sink`, `Source` and `Converter`.

#### Sources

To add sources you basically do the same as for the buses, but in addtition to assigning a label you also specify the bus to which the source is connetected by instantiating a `solph.Flow()` object. 

```{note}
While the classes `Flow()`, `Bus()` and `EnergySystem()` can be imported directly from *solph*. The component classes `Source()`, `Sink()`, `Converter()` and `GenericStorage()` have to be importet from *solph.components*
```

Flows are used to create connections between different elements of your *EnergySystem*. You can connect Components with Buses or Buses with other Buses.
Flows are defined using python dictionaries, where the key is the Bus the *Component* is connected to and the value is `solph.Flow()` object.

```python
{bus: solph.Flow()}
```

On top of creating the topological structure of your *EnergySystem*, the *Flow* holds variables, which are optimized with the solver. This can be the per time step amounts of energy transferred from one part of your system to another one, or the installed capacity of a specific component (e.g. a heat pump). Furthermore, you can set constraints, e.g. limiting upper and lower bounds (constant or time-dependent) or setting summarised limits (such as an annual emission limit). For all parameters see the API documentation of the Flow class. 

If the amount of energy transfered is an optimization variable, you limit its maximum output by using the keyword `nominal_capacity`. Otherwise the amount of energy is limitless.

```{tip}
oemof.solph has no check regarding units, i.e. oemof.solph is not aware if you use [kW], [MW] or something else. To keep track of units, you can write the units in comments behind the value. This also helps debugging later on.
```
To add costs to the energy flow you use the key word `variable costs`, which can be a scalar value or a time series.

```{note}
You can specify prices as *per hour* such as [€/kWh] even if your time resolution is not hourly. oemof-solph will automatically calculate the energy given the power of a unit and the length of the time step specified by the time index of the `EnergySystem()`.
```

In [35]:
el_grid = solph.components.Source(
    label="electricity grid",
    outputs={
        el_bus: solph.Flow(
            nominal_capacity=10,  # max. grid connection capacity in [kW]
            variable_costs=0.3,   # electrictiy price in [€/kWh]
        )
    },
) 

gas_grid = solph.components.Source(
    label="gas grid",
    outputs={
        el_bus: solph.Flow(
            nominal_capacity=10,  # max. grid connection capacity in [kW]
            variable_costs=0.08,   # electrictiy price in [€/kWh]
        )
    },
) 

energy_system.add(el_grid, gas_grid)

If the source is a parameter, i.e. you have a fixed time series for exampel from measurements, you want to use, you use the key word `fix` to indicate this.
You can either use a scalar if the value does not change over time or a time series with the same index as the time index of your energy system.
If your time series or scalar is normalized, you specifiy the absolute value by the `nominal_capacity`. If you time series or scalar is not normalized just specify `nominal_capacity=1`.

In [36]:
import pandas as pd

# read normalized pv yield from csv file
pv_data = pd.read_csv("pv_example_data.csv", index_col=0).iloc[:, 1]

# add pv as a Source to the energy system
pv = solph.components.Source(
    label="pv",
    outputs={
        el_bus: solph.Flow(
            nominal_capacity=12, # pv capacity in [kWp]
            fix=pv_data          # pv yield normalized
        )
    },
)

energy_system.add(pv)

#### Sinks

Sinks work the same as Sources. You just have to define the `inputs` instead of the `outputs`.

In [37]:
el_dem = solph.components.Sink(
    label="electric demand",
    inputs={el_bus: solph.Flow(fix=5, # kW
                               nominal_capacity=1)},
)  #

heat_dem = solph.components.Sink(
    label="heat demand",
    inputs={heat_bus: solph.Flow(fix=5, # kW
                               nominal_capacity=1)}
)  

wb_dem = solph.components.Sink(
    label="wallbox demand",
    inputs={el_bus: solph.Flow(fix=5, # kW
                               nominal_capacity=1)}
)  

feed_in = solph.components.Sink(
    label="feed_in",
    inputs={el_bus: solph.Flow(variable_costs=-0.06)}
) 


energy_system.add(el_dem, heat_dem, wb_dem, feed_in)

#### Converters
An instance of the `Converter` class can represent a node with multiple input and output flows such as a power plant, a transport line or any kind of a transforming process as electrolysis, a cooling device or a heat pump.

For converters you have to define both `inputs` and `outputs`. The dictionary format is especially useful here, since a converter can have multiple input and output flows (sinks and sources can also have mulitple inputs or poutputs respectively).

Converters have a conversion rate that defines the relation between the inputs and outputs. If you are modeling power plant, heat pumps and so on, this is the same as the efficiency of your unit. Conversion factors are given in the form of a dictionary as well.

```
conversion_factors = {input_1: 0.5, input_2: 0.8,
                      output:0.4}
```
This example would translate into the following constraints:

$$
\text{inflow}_{input\_1}\cdot 0.4 = \text{outflow}_{output}\cdot 0.5\\
\text{inflow}_{input\_2}\cdot 0.4 = \text{outflow}_{output}\cdot 0.8\\
$$

If you do not specify a conversion rate it is set to 1.

A simple example for a converter is a gas boiler. By specifing the conversion_factor for the *heat_bus* to be 0.9, you create the desired efficiency: For each kWh of natural gas burned you get 0.9 kWh of heat.

```{note}
As you can see in this example, both heat and gas are expressed in the same unit, and a simple energetic efficiency is used as the conversion factor. You could also model gas in units of m$^3$/s in your model. In that case, the conversion factor would have to account for the volumetric‑to‑energetic conversions as well!
```

In [38]:
gb = solph.components.Converter(
    label="gas boiler",
    inputs={gas_bus: solph.Flow()},
    outputs={heat_bus: solph.Flow(nominal_capacity=5)}, # kW
    conversion_factors={heat_bus: 0.9}, # gas boiler efficiency is 90%
)  
energy_system.add(gb)

In this example the gas boiler has a constant efficiency. You can also define a different efficiency for every time step, e.g. the cop of an air source heat pump changes with the ambient temperatur which itself changes with time. But this time series of the efficiency has to be predefined and cannot be changed within the optimisation. So you would need to calculate the cop of the heat pump for each time step beforehand in a preprocessing step.

In [39]:
ambient_temperature = pd.read_csv("pv_example_data.csv", 
                                  index_col=0).iloc[:, 2]

# approximation of the cop for a supply temperature of 55 °C
cop = (55 + 273.15 * 0.5) / (55 - ambient_temperature)

hp = solph.components.Converter(
    label="heat pump",
    inputs={el_bus: solph.Flow()},
    outputs={heat_bus: solph.Flow(nominal_capacity=7)}, #kW
    conversion_factors={heat_bus: cop},
)

energy_system.add(hp)

An example for a converter with multiple inputs and outputs is a CHP unit:

In [40]:
chp = solph.components.Converter(
    label="chp",
    inputs={gas_bus: solph.Flow()},
    outputs={el_bus: solph.Flow(), heat_bus: solph.Flow(nominal_capacity=8)},
    conversion_factors={el_bus: 0.4, heat_bus: 0.6},
)

energy_system.add(chp)

#### GenericStorages

`GenericStorages` have one input and one output. The `nominal_capacity` of the storage signifies the storage capacity. You can either set it to the net capacity or to the gross capacity and limit it using the minimum/maximum attribute. To limit the input and output flows, you can define the `nominal_capacity` in the Flow objects. If you set the `nominal_capacity` and the `nominal_capacity`of inflow and outflow to the same value, then that is equivalent to a c-rate of 1.

Furthermore, an efficiency for charging and dischaging (`inflow_conversion_factor` and `outflow_conversion_factor`) as well as a `loss_rate` can be defined. The loss rate indicates the amount of energy lost per hour in percent of the current storage content. (If you time step duration is different from hourly, oemof-solph will calculate the losses appropriatly)

The storage content is calculated as:

$$\text{storage content}(t+1) = $$
$$\text{storage content}(t)\cdot (1-\text{loss rate})^{\text{length of time step}}$$
$$+\text{charged energy}(t)\cdot \text{inflow conversion factor} $$
$$- \text{discharged energy}(t) \cdot \frac{1}{\text{outflow conversion factor}}$$

The storage content of the first time step t, is calculated from the parameter `intial_storage_level` which defaults to zero if not specified. The `intial_storage_level` is given as a percentage of the `nominal_capacity` of the storage.

Additionally the parameter `balanced` (default value: `True`) sets the relation of the state of charge of time step zero and the last time step. If `balanced=True`, the state of charge in the last time step is equal to initial value in time step zero (`intital_storage_level`). Use `balanced=False` with caution as energy might be *created* or *destroyed* in the energy system due to different states of charge in time step zero and the last time step. 

In [41]:
bat = solph.components.GenericStorage(
    label="battery",
    inputs={el_bus: solph.Flow(nominal_capacity=6)}, 
    outputs={el_bus: solph.Flow(nominal_capacity=6)},
    nominal_capacity=6,  # kWh
    loss_rate=0.01,
    inflow_conversion_factor=0.98,
    outflow_conversion_factor=0.8,
    initial_storage_level=0.5, 
    balanced=True,
)

energy_system.add(bat)

## Model

At this point, you have added all necessary elements to your *EnergySystem*. It now contains all the information about your system's components, types of energy involved and how everything is connected. 

By doing so, you have also implicitly created the **objective function**: Every time you specified `variable_costs` for a `Flow()`, this flow is automatically added to the objective function with its associated costs. The optimization problem then minimizes the sum of all these contributions:

$$ min \,\, \text{objective} = \sum_n^N \sum_t^T flow(n,t)\cdot \text{variable costs}(n,t)$$

```{note}
The `variable_costs` in oemof.solph do not have to be monetary costs but could also be emissions or other variable units.
```

To solve your energy system optimization model, the *EnergySystem* first needs to be translated into a form that can be processed by a solver such as **CBC** or **Gurobi**. This representation is called an *LP-file* (linear programming file), which contains all equations, constraints, and variables in a standardized mathematical format. oemof-solph does this by using the Python package: [pyomo](https://pyomo.readthedocs.io/en/stable/)

Before solving, your *EnergySystem* is therefore converted into an Pyomo model. You can access this step by creating a `solph.Model()` which takes your `EnergySystem()` class as input.

In [42]:
optimization_model = solph.Model(energy_system)

Now you can solve you model. You can use any solver that is supported by Pyomo. Common choices in energy system modeling include **CBC** (open‑source) and **Gurobi** (academic licence available).

oemof.solph also allows you to pass additional parameters to the solver. For example, you can set a time limit after which the optimization will stop, even if no optimal solution has been found yet. These settings are provided through the `solver_options` argument, which expects a dictionary where the keys are solver‑specific option names and the values are the corresponding parameter values. Because each solver uses slightly different option names, we recommend consulting the documentation for the solver you are using.

If you want to control whether the solver outputs writes logs to the terminal, you can do so with `solve_kwargs={"tee": True}`. Setting `tee` to `True` enables verbose solver output; setting it to `False` suppresses it. This can be helpful when debugging your model.

In [43]:
optimization_model.solve(
    solver="gurobi",
    solver_options={"TimeLimit": 3600},
    solve_kwargs={"tee": True},
)

Set parameter TokenServer to value "10.97.80.117"
Read LP format model from file C:\Users\schev\AppData\Local\Temp\tmp3og6lb_o.pyomo.lp
Reading time = 0.35 seconds
x1: 78841 rows, 122640 columns, 236519 nonzeros
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26200.2))

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 78841 rows, 122640 columns and 236519 nonzeros
Model fingerprint: 0x27c2967f
Coefficient statistics:
  Matrix range     [1e-02, 8e+00]
  Objective range  [6e-02, 3e-01]
  Bounds range     [5e+00, 1e+01]
  RHS range        [1e-02, 1e+01]
Presolve removed 61321 rows and 70511 columns
Presolve time: 0.13s
Presolved: 17520 rows, 52129 columns, 78408 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 1.752e+04
 Factor NZ  : 2.5

{'Problem': [{'Name': 'x1', 'Lower bound': 7978.169478630441, 'Upper bound': 7978.169478630441, 'Number of objectives': 1, 'Number of constraints': 78841, 'Number of variables': 122640, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 122640, 'Number of nonzeros': 236519, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.6789999008178711', 'Error rc': 0, 'Time': 2.0462679862976074}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

Do not be suprised if your first optimization run returns the status **infeasable**. This is a very common occurence when building energy system models. *Infeasable* means that the solver cannot find any solution that satisfies all constraints in your model at the same time. Put simply, somewhere in your model you created an illogical set of equations. 

An example is specifying a maximum output of a source that is lower than a mandatory (fixed) demand at a sink. Since the demand must always be met but the system cannot supply enough energy, no valid solution exists — and the solver reports an infeasible problem.

If your model is infeasable, you have to go on the hunt. The solver (in most cases) can only tell you, *that* your model is infeasable, but not tell you *which* constraint caused the issue. Debugging often involves carefully reviewing your parameter values, bounds, and modeling assumptions.

A related solver message is **unbounded**. This means that the optimization problem has no limit in the direction of the objective — for example, no minimum exists in a minimization problem.

An typical examaple is accidentally assigning negative variable costs to a Source while  forgetting to define a nominal capacity. In such a setup, the model can increase this flow indefinitely, continuously lowering total costs without ever reaching a lower limit.

## Result handling

After solving your model, you currently have two options to access and analyze the results:

1. The established approach using the functions provided in `solph.processing`
2. The experimental `Results()` class, which will become the standard interface in future versions of oemof‑solph

Both options work, but going forward the `Results()` class will be the recommended method.

For an overview how to work with `solph.processing` we refer you to the [tutorial](https://oemof-solph.readthedocs.io/en/stable/basic_concepts/results.html) in the documentation.

In the following you will learn how to use the `Results()` class to access your optimization results.

All optimization results get collected in the instance of `Results().` You can inspect which types of data are available by calling: `Results().keys()`. The available types of data differ slightly between the type of optimization model you are solving and the solver you are using.

In [47]:
results = solph.Results(optimization_model)

keys = results.keys()


print("These are the keys available for this optimization problem:")
for k in keys:
    print(" -", k)


These are the keys available for this optimization problem:
 - flow
 - objective
 - storage_content
 - storage_losses
 - variable_costs
 - Problem
 - Solver
 - Solution


Each key provides access to a different part of the model output. With `Problem`, `Solver` and `Solution` you can access the information about your problem and the solution given back by the solver. These keys come from pyomo and your solver, so they might look differently if you are using a different solver.

**Problem** contains metadata about your optimization problem, such as the number of variables and the number of constraints. This can be useful for comparing models (e.g., runtime differences or structural complexity).

**Solution** contains information about the solve process itself, including: whether the solver terminated successfully, the termination condition and the time required to solve the problem.

**Solution** represents the raw solver output.

In [48]:
problem = results['Problem']
solver = results['Solver']
solution = results['Solution']

info = {'Problem': problem, 
        'Solver': solver,
        'Solution': solution}

for k, v in info.items():
    print(f"{k} returns : --------------")
    print(v)

Problem returns : --------------

- Name: x1
  Lower bound: 7978.169478630441
  Upper bound: 7978.169478630441
  Number of objectives: 1
  Number of constraints: 78841
  Number of variables: 122640
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 122640
  Number of nonzeros: 236519
  Sense: minimize

Solver returns : --------------

- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 0.6789999008178711
  Error rc: 0
  Time: 2.0462679862976074

Solution returns : --------------

- number of solutions: 0
  number of solutions displayed: 0



The remaining keys are provided by oemof.solph and help you access and analyze the actual solution to your optimization problem.

The objective value of your optimization is called by : `results['objective']` 

```{note}
The objective value is equivalent to the total costs of your system, if you use monetary costs. However, if you use different units for costs or add virtual costs to achieve a desired outcome, these are also contained in the objective value and might not reflect the actual system costs!
```

In [63]:
objective = results['objective']
print(f"The objective value is: {objective}")

The objective value is: 7978.1694786293


All optimized instances of `solph.Flow()` in your model can be accessed via the key: `results['flow']`. This returns a single pandas DataFrame containing all time‑series values for every flow in the system. Flows are indexed using a Tuple of the form: `(input_node, output_node)`. You can save this DataFrame to a file (e.g., a CSV) for further processing or analyze it directly in Python.

In [53]:
flow_results = results['flow']
flow_results

electric bus                                            \
                    electric demand wallbox demand feed_in heat pump  battery   
2025-01-01 00:00:00               5              5     0.0  1.357171  0.00000   
2025-01-01 01:00:00               5              5     0.0  1.391100  0.00000   
2025-01-01 02:00:00               5              5     0.0  1.414590  0.00000   
2025-01-01 03:00:00               5              5     0.0  1.417200  0.00000   
2025-01-01 04:00:00               5              5     0.0  1.414590  0.00000   
...                             ...            ...     ...       ...      ...   
2025-12-31 19:00:00               5              5     0.0  1.391100  0.00000   
2025-12-31 20:00:00               5              5     0.0  1.393710  0.00000   
2025-12-31 21:00:00               5              5     0.0  1.406760  0.00000   
2025-12-31 22:00:00               5              5     0.0  1.414590  0.00000   
2025-12-31 23:00:00               5              5     0.0  1.409370  2.84134   

                       heat bus    gas bus      electricity grid     gas grid  \
                    heat demand gas boiler  chp     electric bus electric bus   
2025-01-01 00:00:00           5        0.0  0.0         0.000000         10.0   
2025-01-01 01:00:00           5        0.0  0.0         0.382459         10.0   
2025-01-01 02:00:00           5        0.0  0.0         1.414590         10.0   
2025-01-01 03:00:00           5        0.0  0.0         1.417200         10.0   
2025-01-01 04:00:00           5        0.0  0.0         1.414590         10.0   
...                         ...        ...  ...              ...          ...   
2025-12-31 19:00:00           5        0.0  0.0         1.391100         10.0   
2025-12-31 20:00:00           5        0.0  0.0         1.393710         10.0   
2025-12-31 21:00:00           5        0.0  0.0         1.406760         10.0   
2025-12-31 22:00:00           5        0.0  0.0         1.414590         10.0   
2025-12-31 23:00:00           5        0.0  0.0         4.250709         10.0   

                              pv gas boiler heat pump          chp           \
                    electric bus   heat bus  heat bus electric bus heat bus   
2025-01-01 00:00:00          0.0        0.0       5.0          0.0      0.0   
2025-01-01 01:00:00          0.0        0.0       5.0          0.0      0.0   
2025-01-01 02:00:00          0.0        0.0       5.0          0.0      0.0   
2025-01-01 03:00:00          0.0        0.0       5.0          0.0      0.0   
2025-01-01 04:00:00          0.0        0.0       5.0          0.0      0.0   
...                          ...        ...       ...          ...      ...   
2025-12-31 19:00:00          0.0        0.0       5.0          0.0      0.0   
2025-12-31 20:00:00          0.0        0.0       5.0          0.0      0.0   
2025-12-31 21:00:00          0.0        0.0       5.0          0.0      0.0   
2025-12-31 22:00:00          0.0        0.0       5.0          0.0      0.0   
2025-12-31 23:00:00          0.0        0.0       5.0          0.0      0.0   

                         battery  
                    electric bus  
2025-01-01 00:00:00     1.357171  
2025-01-01 01:00:00     1.008641  
2025-01-01 02:00:00     0.000000  
2025-01-01 03:00:00     0.000000  
2025-01-01 04:00:00     0.000000  
...                          ...  
2025-12-31 19:00:00     0.000000  
2025-12-31 20:00:00     0.000000  
2025-12-31 21:00:00     0.000000  
2025-12-31 22:00:00     0.000000  
2025-12-31 23:00:00     0.000000  

[8760 rows x 16 columns]

You can access a specific flow by providing the `(input_node, output_node)` Tuple:

In [55]:
flow_results[('electric bus', 'electric demand')]

2025-01-01 00:00:00    5
2025-01-01 01:00:00    5
2025-01-01 02:00:00    5
2025-01-01 03:00:00    5
2025-01-01 04:00:00    5
                      ..
2025-12-31 19:00:00    5
2025-12-31 20:00:00    5
2025-12-31 21:00:00    5
2025-12-31 22:00:00    5
2025-12-31 23:00:00    5
Freq: h, Name: (electric bus, electric demand), Length: 8760, dtype: int64

With `storage_content` and `storage_losses` you can access the results of the storage optimization.

In [61]:
storage_results = results['storage_content'], results['storage_losses']

storage_results

(                      battery
 2025-01-01 00:00:00  3.000000
 2025-01-01 01:00:00  1.273536
 2025-01-01 02:00:00  0.000000
 2025-01-01 03:00:00  0.000000
 2025-01-01 04:00:00  0.000000
 ...                       ...
 2025-12-31 20:00:00  0.224327
 2025-12-31 21:00:00  0.222083
 2025-12-31 22:00:00  0.219863
 2025-12-31 23:00:00  0.217664
 2026-01-01 00:00:00  3.000000
 
 [8761 rows x 1 columns],
                       battery
 2025-01-01 00:00:00  0.030000
 2025-01-01 01:00:00  0.012735
 2025-01-01 02:00:00  0.000000
 2025-01-01 03:00:00  0.000000
 2025-01-01 04:00:00  0.000000
 ...                       ...
 2025-12-31 19:00:00  0.002266
 2025-12-31 20:00:00  0.002243
 2025-12-31 21:00:00  0.002221
 2025-12-31 22:00:00  0.002199
 2025-12-31 23:00:00  0.002177
 
 [8760 rows x 1 columns])